# Results Aggregation and Model Comparison

In this notebook, we collect and analyze the performance results of all trained models.  
Each pipeline has been saved as a `.joblib` file during the modeling phase, containing both the fitted model and a structured `results_summary` with detailed evaluation metrics.

The objectives of this notebook are:
- **Aggregate results** from all model families,  
  including both *basic* and *advanced* preprocessing pipelines as well as *TPE* and *GridSearch* optimization.
- **Provide a unified comparison table** across accuracy, precision, recall, F1, F2, and ROC-AUC scores on the final hold-out test set.
- **Visualize key results** with bar plots, confusion matrices, ROC and precision–recall curves, and threshold optimization plots.
- **Support the research questions** by identifying which combinations of preprocessing strategy, representation, and classifier deliver the best balance between performance and complexity.

By consolidating the evaluation in one place, this notebook enables a transparent and reproducible comparison of all tested pipelines. The resulting tables and figures will form the empirical foundation for the discussion and conclusion chapters of the thesis.


In [1]:
from pathlib import Path

root_path = Path.cwd().parents[0]   # supposed project root

print("root_path:", root_path)
print("Does src exist here? ", (root_path / "src").exists())
print("Contents of root_path:", [p.name for p in root_path.iterdir()])
print("Contents of src:", [p.name for p in (root_path / "src").iterdir()] if (root_path / "src").exists() else "no src folder")



root_path: c:\Users\tdoro\Documents\GitHub\clean-repo
Does src exist here?  True
Contents of root_path: ['.git', '.gitattributes', '.gitignore', '.vscode', '01data_collection', '02exploratory_analysis.ipynb', '03prep_for_labeling', '04doccano_labeling', '05modeling_pipelines', '06further_experimentation', '07results', 'base_env_log.yml', 'README.md', 'src', '__init__.py']
Contents of src: ['bilstm_skorch.py', 'cleaning.py', 'eurobert_skorch.py', 'evaluation_visualization.py', 'fasttext_embeddings', 'fasttext_sklearn.py', 'fasttext_vectorizer.py', 'optimize_threshold.py', 'optuna_optimizer.py', 'optuna_visualizations.py', 'phrase_trainer.py', 'phrase_transformer.py', 'preprocessing.py', 'widemlp_skorch.py', '__init__.py', '__pycache__']


In [2]:
from pathlib import Path
import sys
import os
import glob
import joblib
import pandas as pd
import pickle

root_path = Path.cwd().parents[0]  
if str(root_path) not in sys.path:
    sys.path.append(str(root_path))

print("Root path added:", root_path)

import src.optimize_threshold
print("Import works now")

# HuggingFace "transformers_modules" fix
hf_modules_root = Path(r"C:\Users\tdoro\.cache\huggingface\modules")
if str(hf_modules_root) not in sys.path:
    sys.path.append(str(hf_modules_root))
    print("Added HF modules path:", hf_modules_root)

Root path added: c:\Users\tdoro\Documents\GitHub\clean-repo
Import works now
Added HF modules path: C:\Users\tdoro\.cache\huggingface\modules


In [3]:
# path to all model joblibs
models_root = root_path / "05modeling_pipelines"

# search all stored joblib files recursvely
joblib_files = glob.glob(os.path.join(models_root, "**", "*.joblib"), recursive=True)

print(f"Found {len(joblib_files)} joblib files.")
for f in joblib_files:
    print(f)

Found 40 joblib files.
c:\Users\tdoro\Documents\GitHub\clean-repo\05modeling_pipelines\eurobert\model_eurobert_advanced_gridsearch.joblib
c:\Users\tdoro\Documents\GitHub\clean-repo\05modeling_pipelines\eurobert\model_eurobert_advanced_tpe.joblib
c:\Users\tdoro\Documents\GitHub\clean-repo\05modeling_pipelines\eurobert\model_eurobert_basic_gridsearch.joblib
c:\Users\tdoro\Documents\GitHub\clean-repo\05modeling_pipelines\eurobert\model_eurobert_basic_tpe.joblib
c:\Users\tdoro\Documents\GitHub\clean-repo\05modeling_pipelines\fasttext\model_fasttext_advanced_gridsearch.joblib
c:\Users\tdoro\Documents\GitHub\clean-repo\05modeling_pipelines\fasttext\model_fasttext_advanced_tpe.joblib
c:\Users\tdoro\Documents\GitHub\clean-repo\05modeling_pipelines\fasttext\model_fasttext_basic_gridsearch.joblib
c:\Users\tdoro\Documents\GitHub\clean-repo\05modeling_pipelines\fasttext\model_fasttext_basic_tpe.joblib
c:\Users\tdoro\Documents\GitHub\clean-repo\05modeling_pipelines\fasttext_bilstm\model_fasttext_bi

In [4]:
data = joblib.load(f"{models_root}/eurobert/model_eurobert_advanced_tpe.joblib")
print(data.keys())

results_summary = data["results_summary"]

print("Keys in results_summary:", results_summary.keys())
print("Test metrics:", results_summary["test_results"]["test_performance_metrics"])


c:\Users\tdoro\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


dict_keys(['model', 'base_model', 'threshold', 'cv_score', 'best_params', 'search_type', 'results_summary'])
Keys in results_summary: dict_keys(['model_name', 'preprocessing_type', 'optimization_method', 'best_params', 'cv_results', 'test_results', 'dataset_info'])
Test metrics: {'classification_report': {'0': {'precision': 0.9310344827586207, 'recall': 0.9246575342465754, 'f1-score': 0.9278350515463918, 'support': 146.0}, '1': {'precision': 0.8, 'recall': 0.8148148148148148, 'f1-score': 0.8073394495412844, 'support': 54.0}, 'accuracy': 0.895, 'macro avg': {'precision': 0.8655172413793104, 'recall': 0.8697361745306951, 'f1-score': 0.8675872505438381, 'support': 200.0}, 'weighted avg': {'precision': 0.895655172413793, 'recall': 0.895, 'f1-score': 0.8953012390050128, 'support': 200.0}}, 'additional_metrics': {'accuracy': 0.895, 'f2_score': 0.8118081180811808, 'roc_auc': 0.9339167935058346, 'auc_pr': 0.8491080381305212}, 'class_1_metrics': {'precision': 0.8, 'recall': 0.8148148148148148, 

In [5]:
results_summary = data["results_summary"]

print("Keys in results_summary:", results_summary.keys())
print("Test metrics:", results_summary["test_results"]["test_performance_metrics"])

Keys in results_summary: dict_keys(['model_name', 'preprocessing_type', 'optimization_method', 'best_params', 'cv_results', 'test_results', 'dataset_info'])
Test metrics: {'classification_report': {'0': {'precision': 0.9310344827586207, 'recall': 0.9246575342465754, 'f1-score': 0.9278350515463918, 'support': 146.0}, '1': {'precision': 0.8, 'recall': 0.8148148148148148, 'f1-score': 0.8073394495412844, 'support': 54.0}, 'accuracy': 0.895, 'macro avg': {'precision': 0.8655172413793104, 'recall': 0.8697361745306951, 'f1-score': 0.8675872505438381, 'support': 200.0}, 'weighted avg': {'precision': 0.895655172413793, 'recall': 0.895, 'f1-score': 0.8953012390050128, 'support': 200.0}}, 'additional_metrics': {'accuracy': 0.895, 'f2_score': 0.8118081180811808, 'roc_auc': 0.9339167935058346, 'auc_pr': 0.8491080381305212}, 'class_1_metrics': {'precision': 0.8, 'recall': 0.8148148148148148, 'f1_score': 0.8073394495412844}}


In [6]:
import joblib
import pandas as pd
import os, glob, gc, torch

models_root = root_path / "05modeling_pipelines"
joblib_files = glob.glob(os.path.join(models_root, "**", "*.joblib"), recursive=True)

records = []
errors = []

for f in joblib_files:
    try:
        # Laden
        data = joblib.load(f)
        
        # Nur results_summary extrahieren
        summary = data.get("results_summary", {})
        records.append({
            "file": f,
            "model_name": summary.get("model_name"),
            "preprocessing": summary.get("preprocessing_type"),
            "optimization": summary.get("optimization_method"),
            "best_params": summary.get("best_params"),
            "cv_best_f2": summary.get("cv_results", {}).get("best_f2_score"),
            "cv_f2_with_threshold": summary.get("cv_results", {}).get("cv_f2_with_threshold"),
            "test_metrics": summary.get("test_results", {}).get("test_performance_metrics"),
            "threshold": summary.get("test_results", {}).get("threshold_used"),
            "train_size": summary.get("dataset_info", {}).get("train_size"),
            "test_size": summary.get("dataset_info", {}).get("test_size"),
            "train_pos_ratio": summary.get("dataset_info", {}).get("train_positive_ratio"),
            "test_pos_ratio": summary.get("dataset_info", {}).get("test_positive_ratio")
        })
        
        # sofort wieder freigeben
        del data, summary
        gc.collect()
        try:
            torch.cuda.empty_cache()
        except Exception:
            pass
        
    except Exception as e:
        errors.append((f, str(e)))

df_results = pd.DataFrame(records)
print("Loaded summaries from", len(df_results), "joblibs.")
print("Errors:", errors[:3])


Loaded summaries from 40 joblibs.
Errors: []


In [55]:
df_results[["model_name", "test_metrics"]].head()


,model_name,test_metrics
0,eurobert_advanced_gridsearch,{'classification_report': {'0': {'precision': ...
1,eurobert_advanced_tpe,{'classification_report': {'0': {'precision': ...
2,eurobert_basic_gridsearch,{'classification_report': {'0': {'precision': ...
3,eurobert_basic_tpe,{'classification_report': {'0': {'precision': ...
4,fasttext_advanced_gridsearch,{'classification_report': {'0': {'precision': ...


In [56]:
import pandas as pd
from pandas import json_normalize

expanded_rows = []

for _, row in df_results.iterrows():
    metrics = row["test_metrics"] or {}
    
    # 1) Extract classification_report
    cls_report = metrics.get("classification_report", {})
    flat_cls = pd.json_normalize(cls_report, sep="_")
    flat_cls = flat_cls.rename(columns=lambda c: c.replace(" ", "_").replace("-", ""))
    flat_cls = flat_cls.to_dict(orient="records")[0]
    
    # 2) Extract other top-level metrics
    other_metrics = {k: v for k, v in metrics.items() if k not in ["classification_report", "additional_metrics", "class_1_metrics"]}
    
    # 3) Flatten additional_metrics if present
    additional = metrics.get("additional_metrics", {})
    flat_add = pd.json_normalize(additional, sep="_").to_dict(orient="records")[0] if isinstance(additional, dict) else {}
    
    # 4) Flatten class_1_metrics if present
    class1 = metrics.get("class_1_metrics", {})
    flat_class1 = pd.json_normalize(class1, sep="_").to_dict(orient="records")[0] if isinstance(class1, dict) else {}
    
    # 5) Combine everything
    combined = {
        **row.drop("test_metrics").to_dict(),
        **other_metrics,
        **flat_cls,
        **flat_add,
        **flat_class1
    }
    
    expanded_rows.append(combined)

df_expanded = pd.DataFrame(expanded_rows)

print("Available columns:", df_expanded.columns.tolist())
df_expanded.head(3)


Available columns: ['file', 'model_name', 'preprocessing', 'optimization', 'best_params', 'cv_best_f2', 'cv_f2_with_threshold', 'threshold', 'train_size', 'test_size', 'train_pos_ratio', 'test_pos_ratio', 'accuracy', '0_precision', '0_recall', '0_f1score', '0_support', '1_precision', '1_recall', '1_f1score', '1_support', 'macro_avg_precision', 'macro_avg_recall', 'macro_avg_f1score', 'macro_avg_support', 'weighted_avg_precision', 'weighted_avg_recall', 'weighted_avg_f1score', 'weighted_avg_support', 'f2_score', 'roc_auc', 'auc_pr', 'precision', 'recall', 'f1_score']


,file,model_name,preprocessing,optimization,best_params,cv_best_f2,cv_f2_with_threshold,threshold,train_size,test_size,...,weighted_avg_precision,weighted_avg_recall,weighted_avg_f1score,weighted_avg_support,f2_score,roc_auc,auc_pr,precision,recall,f1_score
0,c:\Users\tdoro\Documents\GitHub\clean-repo\05m...,eurobert_advanced_gridsearch,advanced,Optuna TPE/GS + TunedThresholdClassifierCV,"{'clf__lr': 1e-05, 'clf__optimizer__weight_dec...",0.868071,0.895455,0.010714,800,200,...,0.884370,0.885,0.884659,200.0,0.780669,0.933029,0.822920,0.792453,0.777778,0.785047
1,c:\Users\tdoro\Documents\GitHub\clean-repo\05m...,eurobert_advanced_tpe,advanced,Optuna TPE/GS + TunedThresholdClassifierCV,"{'clf__lr': 2.368863950364079e-05, 'clf__batch...",0.840113,0.929809,0.020918,800,200,...,0.895655,0.895,0.895301,200.0,0.811808,0.933917,0.849108,0.800000,0.814815,0.807339
2,c:\Users\tdoro\Documents\GitHub\clean-repo\05m...,eurobert_basic_gridsearch,basic,Optuna TPE/GS + TunedThresholdClassifierCV,"{'clf__lr': 1e-05, 'clf__optimizer__weight_dec...",0.775976,0.883165,0.010459,800,200,...,0.868714,0.860,0.862856,200.0,0.791367,0.903983,0.788090,0.709677,0.814815,0.758621


In [57]:
# relevant columns
cols_keep = [
    "model_name",
    "preprocessing",
    "cv_best_f2",
    "cv_f2_with_threshold",
    "accuracy",
    "f2_score",
    "roc_auc",
    "auc_pr",
    "precision",
    "recall",
    "f1_score"
]

# 
cols_keep = [c for c in cols_keep if c in df_expanded.columns]

df_final_results = df_expanded[cols_keep].copy()

df_final_results

,model_name,preprocessing,cv_best_f2,cv_f2_with_threshold,accuracy,f2_score,roc_auc,auc_pr,precision,recall,f1_score
0,eurobert_advanced_gridsearch,advanced,0.868071,0.895455,0.885,0.780669,0.933029,0.822920,0.792453,0.777778,0.785047
1,eurobert_advanced_tpe,advanced,0.840113,0.929809,0.895,0.811808,0.933917,0.849108,0.800000,0.814815,0.807339
2,eurobert_basic_gridsearch,basic,0.775976,0.883165,0.860,0.791367,0.903983,0.788090,0.709677,0.814815,0.758621
3,eurobert_basic_tpe,basic,0.797552,0.932595,0.830,0.737410,0.903475,0.822520,0.661290,0.759259,0.706897
4,fasttext_advanced_gridsearch,advanced,0.799836,0.737342,0.585,0.754986,0.882927,0.750726,0.392593,0.981481,0.560847
5,fasttext_advanced_tpe,advanced,0.808999,0.646121,0.270,0.649038,0.853501,0.723172,0.270000,1.000000,0.425197
6,fasttext_basic_gridsearch,basic,0.750476,0.750654,0.715,0.772871,0.868087,0.742688,0.485149,0.907407,0.632258
7,fasttext_basic_tpe,basic,0.646121,0.646121,0.270,0.649038,0.512430,0.275041,0.270000,1.000000,0.425197
8,fasttext_bilstm_advanced_tpe,advanced,0.803914,0.766871,0.795,0.731707,0.846651,0.735504,0.591549,0.777778,0.672000
9,fasttext_bilstm_basic_tpe,basic,0.726044,0.738681,0.720,0.730519,0.836124,0.698339,0.489130,0.833333,0.616438


In [58]:

df_final_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   model_name            40 non-null     object 
 1   preprocessing         40 non-null     object 
 2   cv_best_f2            39 non-null     float64
 3   cv_f2_with_threshold  38 non-null     float64
 4   accuracy              40 non-null     float64
 5   f2_score              40 non-null     float64
 6   roc_auc               40 non-null     float64
 7   auc_pr                40 non-null     float64
 8   precision             40 non-null     float64
 9   recall                40 non-null     float64
 10  f1_score              40 non-null     float64
dtypes: float64(9), object(2)
memory usage: 3.6+ KB


In [59]:
# Ensure all model_name values are strings
df_final_results["model_name"] = df_final_results["model_name"].astype(str)
# Clean model_name: ensure string and remove { }, ' and extra spaces
df_final_results["model_name"] = (
    df_final_results["model_name"]
    .astype(str)
    .str.replace(r"[{}']", "", regex=True)  # remove {, }, and '
    .str.strip()
)

# sanity check: no braces or quotes left
assert not df_final_results["model_name"].str.contains(r"[{}']").any(), "Braces or quotes still present!"

# Extract suffix after last "_"
df_final_results["hp_search"] = df_final_results["model_name"].str.split("_").str[-1]

# Map suffix to normalized values
df_final_results["hp_search"] = df_final_results["hp_search"].map({
    "gridsearch": "gridsearch",
    "tpe": "optuna_tpe"
})

df_final_results

,model_name,preprocessing,cv_best_f2,cv_f2_with_threshold,accuracy,f2_score,roc_auc,auc_pr,precision,recall,f1_score,hp_search
0,eurobert_advanced_gridsearch,advanced,0.868071,0.895455,0.885,0.780669,0.933029,0.822920,0.792453,0.777778,0.785047,gridsearch
1,eurobert_advanced_tpe,advanced,0.840113,0.929809,0.895,0.811808,0.933917,0.849108,0.800000,0.814815,0.807339,optuna_tpe
2,eurobert_basic_gridsearch,basic,0.775976,0.883165,0.860,0.791367,0.903983,0.788090,0.709677,0.814815,0.758621,gridsearch
3,eurobert_basic_tpe,basic,0.797552,0.932595,0.830,0.737410,0.903475,0.822520,0.661290,0.759259,0.706897,optuna_tpe
4,fasttext_advanced_gridsearch,advanced,0.799836,0.737342,0.585,0.754986,0.882927,0.750726,0.392593,0.981481,0.560847,gridsearch
5,fasttext_advanced_tpe,advanced,0.808999,0.646121,0.270,0.649038,0.853501,0.723172,0.270000,1.000000,0.425197,optuna_tpe
6,fasttext_basic_gridsearch,basic,0.750476,0.750654,0.715,0.772871,0.868087,0.742688,0.485149,0.907407,0.632258,gridsearch
7,fasttext_basic_tpe,basic,0.646121,0.646121,0.270,0.649038,0.512430,0.275041,0.270000,1.000000,0.425197,optuna_tpe
8,fasttext_bilstm_advanced_tpe,advanced,0.803914,0.766871,0.795,0.731707,0.846651,0.735504,0.591549,0.777778,0.672000,optuna_tpe
9,fasttext_bilstm_basic_tpe,basic,0.726044,0.738681,0.720,0.730519,0.836124,0.698339,0.489130,0.833333,0.616438,optuna_tpe


In [60]:
import pandas as pd

# numeric columns we want to compare column-wise
numeric_cols = [
    "cv_best_f2","cv_f2_with_threshold","accuracy","f2_score",
    "roc_auc","auc_pr","precision","recall","f1_score"
]

# ensure numeric dtypes
df_final_results[numeric_cols] = df_final_results[numeric_cols].apply(
    pd.to_numeric, errors="coerce"
)

# 1) Show in notebook with bold column-wise maxima
styled = (
    df_final_results
    .style
    # highlight column-wise maxima with bold font
    .highlight_max(subset=numeric_cols, axis=0, props="font-weight: bold;")
    # optional: consistent number formatting
    .format({c: "{:.3f}" for c in numeric_cols})
)
styled  # renders in Jupyter


,model_name,preprocessing,cv_best_f2,cv_f2_with_threshold,accuracy,f2_score,roc_auc,auc_pr,precision,recall,f1_score,hp_search
0,eurobert_advanced_gridsearch,advanced,0.868,0.895,0.885,0.781,0.933,0.823,0.792,0.778,0.785,gridsearch
1,eurobert_advanced_tpe,advanced,0.840,0.930,0.895,0.812,0.934,0.849,0.800,0.815,0.807,optuna_tpe
2,eurobert_basic_gridsearch,basic,0.776,0.883,0.860,0.791,0.904,0.788,0.710,0.815,0.759,gridsearch
3,eurobert_basic_tpe,basic,0.798,0.933,0.830,0.737,0.903,0.823,0.661,0.759,0.707,optuna_tpe
4,fasttext_advanced_gridsearch,advanced,0.800,0.737,0.585,0.755,0.883,0.751,0.393,0.981,0.561,gridsearch
5,fasttext_advanced_tpe,advanced,0.809,0.646,0.270,0.649,0.854,0.723,0.270,1.000,0.425,optuna_tpe
6,fasttext_basic_gridsearch,basic,0.750,0.751,0.715,0.773,0.868,0.743,0.485,0.907,0.632,gridsearch
7,fasttext_basic_tpe,basic,0.646,0.646,0.270,0.649,0.512,0.275,0.270,1.000,0.425,optuna_tpe
8,fasttext_bilstm_advanced_tpe,advanced,0.804,0.767,0.795,0.732,0.847,0.736,0.592,0.778,0.672,optuna_tpe
9,fasttext_bilstm_basic_tpe,basic,0.726,0.739,0.720,0.731,0.836,0.698,0.489,0.833,0.616,optuna_tpe


In [61]:
import pandas as pd
from datetime import datetime

numeric_cols = [
    "cv_best_f2","cv_f2_with_threshold","accuracy","f2_score",
    "roc_auc","auc_pr","precision","recall","f1_score"
]
df_final_results[numeric_cols] = df_final_results[numeric_cols].apply(
    pd.to_numeric, errors="coerce"
)
# Round all float columns to 3 decimals
df_final_results = df_final_results.round(3)


styled = (
    df_final_results
    .style
    .highlight_max(subset=numeric_cols, axis=0, props="font-weight: bold;")
    .format({c: "{:.3f}" for c in numeric_cols})
)

out_path = f"model_results_{datetime.now():%Y%m%d_%H%M}.xlsx"
with pd.ExcelWriter(out_path, engine="openpyxl") as writer:
    styled.to_excel(writer, sheet_name="Results", index=False)

print("Exported:", out_path)


Exported: model_results_20250917_1856.xlsx


In [ ]:
# Copy dataframe to avoid inplace modification
df_final_results

# Adjust model_name directly
df_final_results.loc[10, "model_name"] = df_final_results.loc[10, "model_name"].replace("tfidf_bilstm", "fasttext_bilstm")
df_final_results.loc[11, "model_name"] = df_final_results.loc[11, "model_name"].replace("tfidf_bilstm", "fasttext_bilstm")
df_final_results.loc[18, "model_name"] = df_final_results.loc[18, "model_name"].replace("tfidf_xgb", "fasttext_xgb")

# Check results
print(df_final_results.loc[[10, 11, 18], "model_name"])


In [64]:
import re
#map base model
def extract_model_base(name: str) -> str:
    # End-to-End models
    if name.startswith("eurobert"):
        return "eurobert"
    if name.startswith("fasttext_basic") or name.startswith("fasttext_advanced"):
        return "fasttext"
    if name.startswith("baseline"):
        return "baseline"
    
    # All other pipelines → keep embedding+classifier (remove basic/advanced/tpe/gridsearch)
    return re.sub(r"_(basic|advanced)_(tpe|gridsearch)$", "", name)




df_final_results["model_base"] = df_final_results["model_name"].apply(extract_model_base)
df_final_results


,model_name,preprocessing,cv_best_f2,cv_f2_with_threshold,accuracy,f2_score,roc_auc,auc_pr,precision,recall,f1_score,hp_search,model_base
0,eurobert_advanced_gridsearch,advanced,0.868,0.895,0.885,0.781,0.933,0.823,0.792,0.778,0.785,gridsearch,eurobert
1,eurobert_advanced_tpe,advanced,0.840,0.930,0.895,0.812,0.934,0.849,0.800,0.815,0.807,optuna_tpe,eurobert
2,eurobert_basic_gridsearch,basic,0.776,0.883,0.860,0.791,0.904,0.788,0.710,0.815,0.759,gridsearch,eurobert
3,eurobert_basic_tpe,basic,0.798,0.933,0.830,0.737,0.903,0.823,0.661,0.759,0.707,optuna_tpe,eurobert
4,fasttext_advanced_gridsearch,advanced,0.800,0.737,0.585,0.755,0.883,0.751,0.393,0.981,0.561,gridsearch,fasttext
5,fasttext_advanced_tpe,advanced,0.809,0.646,0.270,0.649,0.854,0.723,0.270,1.000,0.425,optuna_tpe,fasttext
6,fasttext_basic_gridsearch,basic,0.750,0.751,0.715,0.773,0.868,0.743,0.485,0.907,0.632,gridsearch,fasttext
7,fasttext_basic_tpe,basic,0.646,0.646,0.270,0.649,0.512,0.275,0.270,1.000,0.425,optuna_tpe,fasttext
8,fasttext_bilstm_advanced_tpe,advanced,0.804,0.767,0.795,0.732,0.847,0.736,0.592,0.778,0.672,optuna_tpe,fasttext_bilstm
9,fasttext_bilstm_basic_tpe,basic,0.726,0.739,0.720,0.731,0.836,0.698,0.489,0.833,0.616,optuna_tpe,fasttext_bilstm
